# Make an EPN-TAP data table for PLANMAP packages

We want to go inside Planmap packages and extract information for each data product in `vector` and `raster` directories.
This information will be used, initially, to populate VESPA EPN-TAP tables.
In particular, we're interested in filling the following columns<sup>*</sup>:

* `access_url`: data product URL
* `c1max`: max longitude (positive east, 0-360)
* `c1min`: min longitude (positive east, 0-360)
* `c2max`: max latitude
* `c2min`: min latitude
* `s_region`: map footprint
* `creation_date`: package creation date
* `dataproduct_type`: options are `im`
* `granule_gid`: package-id (PM_ID)
* `granule_uid`: data product name
* `modification_date`: package update date (same as creation data, if in doubt)
* `obs_id`: data product name
* `release_date`: package release date (same as creation date, if in doubt)
* `service_title`: `planmap`
* `spatial_frame_type`: `body`
* `target_class`: `planet`
* `target_name`: options are `mars`, `mercury`, `moon`
* `thumbnail_url`: thumbnail (jpeg) url

<sup>*</sup> About EPN-TAP columns: https://voparis-wiki.obspm.fr/display/VES/EPN-TAP+v2+parameter+description

### Software/libraries we're using here:

* Python 3.7.8
* rasterio 1.2.0
* shapely 1.7.1
* fiona 1.8.18

In [1]:
import pandas
pandas.set_option('display.max_colwidth',100)
pandas.set_option('display.max_rows', 100)

In [2]:
from dataclasses import make_dataclass, field, asdict

epntap_fields = [
    ('granule_uid', str),
    ('granule_gid', str),
    ('obs_id', str),
    ('c1min', float, field(default=0)),
    ('c1max', float, field(default=360)),
    ('c2min', float, field(default=-90)),
    ('c2max', float, field(default=90)),
    ('s_region', str, field(default='')),
    ('dataproduct_type', str, field(default='')),
    ('access_url', str, field(default='')),
    ('creation_date', str, field(default='2019-12-31')),
    ('modification_date', str, field(default='2019-12-31')),
    ('release_date', str, field(default='2019-12-31')),
    ('service_title', str, field(default='planmap')),
    ('spatial_frame_type', str, field(default='body')),
    ('target_class', str, field(default='planet')),
    ('target_name', str, field(default='')),
    ('thumbnail_url', str, field(default=''))
]

EPNTAPRecord = make_dataclass('EPNTAPRecord', epntap_fields)

EPNTAPRecordRaster = make_dataclass('EPNTAPRecordRaster', 
                                    [('dataproduct_type', str, 'im')],
                                    bases=(EPNTAPRecord,))

EPNTAPRecordVector = make_dataclass('EPNTAPRecordVector', 
                                    [('dataproduct_type', str, 'sv')],
                                    bases=(EPNTAPRecord,))

In [23]:
import pathlib

import fiona
import shapely
import rasterio


crs = {
    'moon': '+proj=longlat +a=1737400 +b=1737400 +no_defs',
    'mars': '+proj=longlat +a=3396190 +b=3376200 +no_defs',
    'mercury': '+proj=longlat +a=2439700 +b=2439700 +no_defs '
}

def bbox_shift_0360(bbox):
    c1min = bbox['c1min'] + 360 if bbox['c1min'] < 0 else bbox['c1min']
    c1max = bbox['c1max'] + 360 if bbox['c1max'] < 0 else bbox['c1max']
    bbox['c1min'] = c1min
    bbox['c1max'] = c1max
    return bbox
    
def bbox_shift_180180(bbox):
    c1min = bbox['c1min'] if bbox['c1min'] < 180 else bbox['c1min'] - 360
    c1max = bbox['c1max'] if bbox['c1max'] < 180 else bbox['c1max'] - 360
    bbox['c1min'] = c1min
    bbox['c1max'] = c1max
    return bbox
    
def bounding_box_raster(tif, body):
    from rasterio import warp
    img = rasterio.open(tif)
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(img.crs, dst_crs, *img.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    #return bbox_shift_0360(bbox)
    return bbox_shift_180180(bbox)
    
def bounding_box_vector(gpkg, body):
    from rasterio import warp
    f = fiona.open(gpkg)
    dst_crs = crs[body]
    left, bottom, right, top = warp.transform_bounds(f.crs, dst_crs, *f.bounds)
    bbox = {
        'c1min': left,
        'c1max': right,
        'c2min': bottom,
        'c2max': top
    }
    #return bbox_shift_0360(bbox)
    return bbox_shift_180180(bbox)

def s_region_bbox(bbox):
    from shapely import geometry
    coords = [
        (bbox['c1min'],bbox['c2min']),
        (bbox['c1min'],bbox['c2max']),
        (bbox['c1max'],bbox['c2max']),
        (bbox['c1max'],bbox['c2min']),
    ]
    pol = geometry.Polygon(coords)
    return str(pol).replace('(','').replace(')','').replace(',','').replace('POLYGON','Polygon UNKNOWNFrame')

def init_path(path):
    p = pathlib.Path(path)
    if not (p.exists() and p.is_dir()):
        return None
    return p
    
    
class PmPkg:
    def __init__(self, path):
        _path = init_path(path)
        assert _path is not None, "Path '{!s}' does not seem to exist".format(path)
        self._path = _path
        self._pmid = _path.name
        self._body = None
        
    @property
    def pmid(self):
        return self._pmid
    
    def parse_readme(self, param):
        _files = self._path.glob('*.*')
        for _file in _files:
            if _file.suffix.lower() == '.md':
                break
        assert _file
        with open(_file, 'r') as fp:
            _lines = fp.readlines()
        for _l in _lines:
            _l = _l.strip().lower()
            if param.lower() in _l:
                return _l.split('|')[2].strip()
        return ''
        
    def proc_raster_dir(self):
        rasters = []
        path = self._path / 'raster'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix == '.tif' or _f.suffix == '.tiff':
                    target_name = self.parse_readme('target body')
                    assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
#                     thumbnail_url = '.'.join(access_url.split('.')[:-1] + ['jpg'])
                    bbox = bounding_box_raster(_f, target_name)
                    raster = self.make_epntap_record(_f, bbox, target_name)
                    raster['dataproduct_type'] = 'im'
                    rasters.append(raster)
        return rasters
        
    def proc_vector_dir(self):
        vectors = []
        path = self._path / 'vector'
        if path.exists and path.is_dir():
            for _f in path.iterdir():
                if _f.suffix != '.gpkg':
                    continue
                target_name = self.parse_readme('target body')
                assert target_name in ('mars', 'mercury', 'moon'), "Expecting 'target_name' in 'mars', 'mercury', or 'moon'"
                bbox = bounding_box_vector(_f, target_name)
                vector = self.make_epntap_record(_f, bbox, target_name)
                vector['dataproduct_type'] = 'sv'
                vectors.append(vector)
        return vectors                
    
    def proc_readme(self):
        def read_readme_table(readme_path):
            """
            Read README table into a dictionary
            """
            from string import ascii_letters
            out = {}
            with open(readme_path, 'r') as fp:
                lines = fp.readlines()
                for line in lines:
                    line = line.strip()
                    if len(line) == 0 or line[0] != '|':
                        continue
                    fields = line.strip('|').split('|')
                    if len(fields) < 2:
                        continue
                    k,v = [f.strip() for f in fields[:2]]
                    if k[0] in ascii_letters:
                        out[k] = v
            return out
        
        _files = self._path.glob('*.*')
        for _file in _files:
            if _file.suffix.lower() == '.md':
                break
        readme_table = read_readme_table(_file.as_posix())
        bbox = {
            'c1min' : float(readme_table['Bounding box - Min Lon (0-360)']),
            'c1max' : float(readme_table['Bounding box - Max Lon (0-360)']),
            'c2min' : float(readme_table['Bounding box - Min Lat']),
            'c2max' : float(readme_table['Bounding box - Max Lat'])
        }
        target_name = readme_table['Target body'].lower()
        _docs = _file.parent / 'document'
        file_map = _docs / (self.pmid + '.pdf')
        record = self.make_epntap_record(file_map, bbox, target_name)
        return record
        
    def make_epntap_record(self, file, bbox, target_name):
        granule_uid = file.stem
        granule_gid = self.pmid
        obs_id = granule_uid
        
        assert target_name in ('mars', 'mercury', 'moon'), """Expecting 'target_name' in 
                                                            'mars', 'mercury', or 'moon'"""
        self._body = target_name
        
        access_url = '/'.join(['https://data.planmap.eu/pub', 
                               target_name, self.pmid, file.parent.name, file.name])

        record = dict(granule_uid = granule_uid,
                      obs_id = obs_id,
                      granule_gid = granule_gid,
                      access_url = access_url,
                      target_name = target_name,
                      c1min = bbox['c1min'],
                      c1max = bbox['c1max'],
                      c2min = bbox['c2min'],
                      c2max = bbox['c2max'],
                      s_region = s_region_bbox(bbox)
                    )
        return record
        
    def proc_data(self):
        records = []
        rasters = self.proc_raster_dir()
        records.extend(rasters)
        vectors = self.proc_vector_dir()
        records.extend(vectors)
        if not (rasters or vectors):
            readme = self.proc_readme()
            records.append(readme)
        self._records = records

    def to_epntap(self):
        return self._records
    

In [24]:
pkg = PmPkg('./PM-MAR-C-Arsinoes/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.469381,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.469381,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im


In [25]:
pkg = PmPkg('./PM-MAR-MS-Arsinoes/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-MS-Arsinoes/raster/PM-MAR-MS-Arsinoes.tiff,mars,-30.248356,-25.568330,-12.003663,-5.749317,Polygon UNKNOWNFrame -30.24835590928195 -12.00366265420823 -30.24835590928195 -5.749316616352282...,im
1,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,PM-MAR-MS-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-MS-Arsinoes/vector/PM-MAR-MS-Arsinoes.gpkg,mars,-30.283217,-25.483858,-12.084578,-5.815578,Polygon UNKNOWNFrame -30.28321743919844 -12.0845776589575 -30.28321743919844 -5.815578111214499 ...,sv


In [26]:
pkg = PmPkg('./PM-MAR-D-Crommelin/')

pkg.proc_data()

epntap_records = pkg.to_epntap()

import pandas

pandas.DataFrame(epntap_records)

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region
0,PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,PM-MAR-D-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-D-Crommelin/document/PM-MAR-D-Crommelin.pdf,mars,348.8477,350.98821,3.96993,6.22761,Polygon UNKNOWNFrame 348.8477 3.96993 348.8477 6.22761 350.98821 6.22761 350.98821 3.96993 348.8...


In [27]:
import json

from IPython.display import display

import pathlib
cwd = pathlib.Path('.')

def summary_package(df,pmid):
    """
    This is a hack for planmap maps.planmap.eu app,
    I want to profit fom this processing and get the bounding-box in [-180:180] to put there.
    """
    if not len(df):
        return False
    name = '-'.join(pmid.split('-')[-2:])
    
    c1min = df.c1min.min()
    c1min = c1min if c1min < 180 else c1min - 360
    c1max = df.c1max.max()
    c1max = c1max if c1max < 180 else c1max - 360
    c2min = df.c2min.min()
    c2max = df.c2max.max()
    
    bla = {
        'name' : name,
        'pm_id' : pmid,
        'center' : dict(
            lon= (c1min + c1max)/2,
            lat= (c2min + c2max)/2
        ),
        'bbox' : dict(
            xmin= c1min,
            xmax= c1max,
            ymin= c2min,
            ymax= c2max,
            srs= 'EPSG:4326'
        )
    }
    return bla
    
    
df = None

summaries = {
    'mars':[],
    'mercury':[],
    'moon':[]
}

packages = list(cwd.iterdir())
packages.sort()
for _dir in packages:
    if not (_dir.is_dir() and _dir.name[:2] == 'PM'):
        continue
        
    print('#---#---#')
    print(F"Package: {_dir}")
    try:
        pkg = PmPkg(_dir)
        pkg.proc_data()
        epntap_records = pkg.to_epntap()
    except Exception as err:
        print(err)
    _df = pandas.DataFrame(epntap_records)
    _sum = summary_package(_df, _dir.name)
    if not _sum:
        print("FAILED")
        continue
    summaries[pkg._body].append(_sum)
    print(json.dumps(_sum, indent=2))
    display(_df)
    print('#---.---.')

    if df is None:
        df = _df
    else:
        df = pandas.concat([df,_df]).reset_index(drop=True)

#---#---#
Package: PM-MAR-C-Arsinoes
{
  "name": "C-Arsinoes",
  "pm_id": "PM-MAR-C-Arsinoes",
  "center": {
    "lon": -28.35494805881746,
    "lat": -13.0749921424451
  },
  "bbox": {
    "xmin": -28.46938117420382,
    "xmax": -28.240514943431098,
    "ymin": -13.169841214236444,
    "ymax": -12.980143070653755,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.469381,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.469381,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-28.465849,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im


#---.---.
#---#---#
Package: PM-MAR-C-Crommelin
{
  "name": "C-Crommelin",
  "pm_id": "PM-MAR-C-Crommelin",
  "center": {
    "lon": -10.309911318473514,
    "lat": 7.279592356400499
  },
  "bbox": {
    "xmin": -10.844780922334412,
    "xmax": -9.775041714612618,
    "ymin": 4.39276139195179,
    "ymax": 10.166423320849209,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frt0001fd59...,mars,-10.610092,-10.374129,9.959918,10.166423,Polygon UNKNOWNFrame -10.61009214628938 9.959917799011103 -10.61009214628938 10.16642332084921 -...,im
1,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029e82...,mars,-10.823446,-10.617244,10.038756,10.114374,Polygon UNKNOWNFrame -10.82344594864566 10.03875622022168 -10.82344594864566 10.11437368336533 -...,im
2,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029fb5...,mars,-10.032555,-9.775042,4.897460,5.001011,Polygon UNKNOWNFrame -10.03255462716689 4.897459599618517 -10.03255462716689 5.001011372237628 -...,im
3,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt0001fd59_07_...,mars,-10.610092,-10.374129,9.959918,10.166423,Polygon UNKNOWNFrame -10.61009214628938 9.959917799011103 -10.61009214628938 10.16642332084921 -...,im
4,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrl000064b2_07_...,mars,-10.484152,-10.222995,4.392761,4.800288,Polygon UNKNOWNFrame -10.48415183954966 4.39276139195179 -10.48415183954966 4.800287722904422 -1...,im
5,PM-MAR-C-Crommelin_01_frt00017c94_07_if165s_trr3_TRU_1-99,PM-MAR-C-Crommelin_01_frt00017c94_07_if165s_trr3_TRU_1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt00017c94_07_...,mars,-10.844781,-10.603363,4.668037,4.889413,Polygon UNKNOWNFrame -10.84478092233441 4.668037280008146 -10.84478092233441 4.889413356956334 -...,im
6,PM-MAR-C-Crommelin_01_frs00029fb5_01_if167s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frs00029fb5_01_if167s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frs00029fb5_01_...,mars,-10.032555,-9.775042,4.897460,5.001011,Polygon UNKNOWNFrame -10.03255462716689 4.897459599618517 -10.03255462716689 5.001011372237628 -...,im
7,PM-MAR-C-Crommelin_01_FEM_hrs00012a18_07_if174s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_hrs00012a18_07_if174s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_hrs00012a18...,mars,-10.123769,-9.915451,4.884428,5.039908,Polygon UNKNOWNFrame -10.12376880677268 4.8844284373811 -10.12376880677268 5.03990793175349 -9.9...,im
8,PM-MAR-C-Crommelin_01_hrs00012a18_07_if174s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrs00012a18_07_if174s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrs00012a18_07_...,mars,-10.123769,-9.915451,4.884428,5.039908,Polygon UNKNOWNFrame -10.12376880677268 4.8844284373811 -10.12376880677268 5.03990793175349 -9.9...,im
9,PM-MAR-C-Crommelin_01_FEM_hrl000064b2_07_if182s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_hrl000064b2_07_if182s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_hrl000064b2...,mars,-10.484152,-10.222995,4.392761,4.800288,Polygon UNKNOWNFrame -10.48415183954966 4.39276139195179 -10.4

#---.---.
#---#---#
Package: PM-MAR-D-Crommelin
{
  "name": "D-Crommelin",
  "pm_id": "PM-MAR-D-Crommelin",
  "center": {
    "lon": -10.082045000000022,
    "lat": 5.09877
  },
  "bbox": {
    "xmin": -11.152300000000025,
    "xmax": -9.011790000000019,
    "ymin": 3.96993,
    "ymax": 6.22761,
    "srs": "EPSG:4326"
  }
}


...

#---.---.
#---#---#
Package: PM-MOO-MS-Copernicus
CRS is invalid: None
{
  "name": "MS-Copernicus",
  "pm_id": "PM-MOO-MS-Copernicus",
  "center": {
    "lon": -23.0,
    "lat": -3.0
  },
  "bbox": {
    "xmin": -28.0,
    "xmax": -18.0,
    "ymin": -8.0,
    "ymax": 2.0,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region
0,PM-MOO-MS-Apollo12,PM-MOO-MS-Apollo12,PM-MOO-MS-Apollo12,https://data.planmap.eu/pub/moon/PM-MOO-MS-Apollo12/document/PM-MOO-MS-Apollo12.pdf,moon,332.0,342.0,-8.0,2.0,Polygon UNKNOWNFrame 332 -8 332 2 342 2 342 -8 332 -8


#---.---.
#---#---#
Package: PM-MOO-MS-SPAApollo
{
  "name": "MS-SPAApollo",
  "pm_id": "PM-MOO-MS-SPAApollo",
  "center": {
    "lon": 0.1375830224489647,
    "lat": -34.17388959795346
  },
  "bbox": {
    "xmin": -179.45420444093048,
    "xmax": 179.7293704858284,
    "ymin": -61.710997049914084,
    "ymax": -6.636782145992844,
    "srs": "EPSG:4326"
  }
}


,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MOO-MS-SPAApollo_LRO_LOLA_shaded,PM-MOO-MS-SPAApollo_LRO_LOLA_shaded,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/raster/PM-MOO-MS-SPAApollo_LRO_LOLA_shaded.tif,moon,-179.454204,179.729370,-61.710997,-6.636782,Polygon UNKNOWNFrame -179.4542044409305 -61.71099704991408 -179.4542044409305 -6.636782145992844...,im
1,PM-MOO-MS-SPAApollo_LRO_LOLA,PM-MOO-MS-SPAApollo_LRO_LOLA,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/raster/PM-MOO-MS-SPAApollo_LRO_LOLA.tif,moon,-179.454204,179.729370,-61.710997,-6.636782,Polygon UNKNOWNFrame -179.4542044409305 -61.71099704991408 -179.4542044409305 -6.636782145992844...,im
2,PM-MOO-MS-SPAApollo,PM-MOO-MS-SPAApollo,PM-MOO-MS-SPAApollo,https://data.planmap.eu/pub/moon/PM-MOO-MS-SPAApollo/vector/PM-MOO-MS-SPAApollo.gpkg,moon,-164.082509,-140.366787,-44.070362,-25.836114,Polygon UNKNOWNFrame -164.0825091451837 -44.07036239454467 -164.0825091451837 -25.83611438629649...,sv


#---.---.


In [28]:
import json
for body,sums in summaries.items():
    print('-'*len(body))
    print(body)
    print('-'*len(body))
    for _sum in sums:
        print(json.dumps(_sum, indent=2))
    print()

----
mars
----
{
  "name": "C-Arsinoes",
  "pm_id": "PM-MAR-C-Arsinoes",
  "center": {
    "lon": -28.35494805881746,
    "lat": -13.0749921424451
  },
  "bbox": {
    "xmin": -28.46938117420382,
    "xmax": -28.240514943431098,
    "ymin": -13.169841214236444,
    "ymax": -12.980143070653755,
    "srs": "EPSG:4326"
  }
}
{
  "name": "C-Crommelin",
  "pm_id": "PM-MAR-C-Crommelin",
  "center": {
    "lon": -10.309911318473514,
    "lat": 7.279592356400499
  },
  "bbox": {
    "xmin": -10.844780922334412,
    "xmax": -9.775041714612618,
    "ymin": 4.39276139195179,
    "ymax": 10.166423320849209,
    "srs": "EPSG:4326"
  }
}
{
  "name": "D-Crommelin",
  "pm_id": "PM-MAR-D-Crommelin",
  "center": {
    "lon": -10.082045000000022,
    "lat": 5.09877
  },
  "bbox": {
    "xmin": -11.152300000000025,
    "xmax": -9.011790000000019,
    "ymin": 3.96993,
    "ymax": 6.22761,
    "srs": "EPSG:4326"
  }
}
{
  "name": "D-Gale_Kimberley",
  "pm_id": "PM-MAR-D-Gale_Kimberley",
  "center": {
    "l

In [29]:
df

,granule_uid,obs_id,granule_gid,access_url,target_name,c1min,c1max,c2min,c2max,s_region,dataproduct_type
0,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_FEM,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-2.846938e+01,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
1,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_CHL,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-2.846585e+01,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
2,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes_frt00008233_07_if164s_trr3_TRU,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-2.846938e+01,-28.241921,-13.169841,-12.980949,Polygon UNKNOWNFrame -28.46938117420382 -13.16984121423644 -28.46938117420382 -12.98094940215479...,im
3,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_HYD,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-2.846585e+01,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
4,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes_frt00008233_07_if164l_trr3_MAF,PM-MAR-C-Arsinoes,https://data.planmap.eu/pub/mars/PM-MAR-C-Arsinoes/raster/PM-MAR-C-Arsinoes_frt00008233_07_if164...,mars,-2.846585e+01,-28.240515,-13.169642,-12.980143,Polygon UNKNOWNFrame -28.46584890575358 -13.16964225190609 -28.46584890575358 -12.98014307065375...,im
5,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frt0001fd59_07_if165s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frt0001fd59...,mars,-1.061009e+01,-10.374129,9.959918,10.166423,Polygon UNKNOWNFrame -10.61009214628938 9.959917799011103 -10.61009214628938 10.16642332084921 -...,im
6,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029e82_01_if169s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029e82...,mars,-1.082345e+01,-10.617244,10.038756,10.114374,Polygon UNKNOWNFrame -10.82344594864566 10.03875622022168 -10.82344594864566 10.11437368336533 -...,im
7,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin_01_FEM_frs00029fb5_01_if167s_trr3_VNIR_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_FEM_frs00029fb5...,mars,-1.003255e+01,-9.775042,4.897460,5.001011,Polygon UNKNOWNFrame -10.03255462716689 4.897459599618517 -10.03255462716689 5.001011372237628 -...,im
8,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_frt0001fd59_07_if165s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_frt0001fd59_07_...,mars,-1.061009e+01,-10.374129,9.959918,10.166423,Polygon UNKNOWNFrame -10.61009214628938 9.959917799011103 -10.61009214628938 10.16642332084921 -...,im
9,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin_01_hrl000064b2_07_if182s_trr3_TRU_st1-99,PM-MAR-C-Crommelin,https://data.planmap.eu/pub/mars/PM-MAR-C-Crommelin/raster/PM-MAR-C-Crommelin_01_hrl000064b2_07_...,mars,-1.048415e+01,-10.222995,4.392761,4.800288,Polygon UNKNOWNFrame -10.48415183954966 4.39276139195179 -10.48415183954966 4.800287722904422 -1...,im


In [30]:
df.to_csv('/Users/chbrandt/Coisas/repos/VESPA/services/planmap/data/data.csv')